In [2]:
import pandas as pd
from collections import defaultdict
import os
%cd /cluster/home/futing/Project/GBM/HiC/HAR/HARs_gene/genes/long

/cluster/home/futing/Project/GBM/HiC/HAR/HARs_gene/genes/long


In [3]:
HARs_gene=pd.read_csv('/cluster/home/futing/Project/GBM/HiC/HAR/HARs_gene/genes/all/HARs_gene.txt',sep='\t')
HARs_gene_long = HARs_gene.melt(id_vars=['HAR'], value_vars=['GBM', 'NPC', 'iPSC', 'NHA'], 
	var_name='cell', value_name='gene')
HARs_gene_long = HARs_gene_long.dropna()
HARs_gene_long.rename(columns={'HAR':'enhancer'}, inplace=True)

In [4]:
HARs_gene_long.head()

,enhancer,cell,gene
1,chr10_118043738_118044037,GBM,EMX2
2,chr10_118043738_118044037,GBM,EMX2OS
4,chr10_24728545_24728698,GBM,ENSG00000228667
5,chr10_24728545_24728698,GBM,ENSG00000273107
6,chr10_24728545_24728698,GBM,ENSG00000285859


In [9]:
df = HARs_gene_long.loc[HARs_gene_long['cell'].isin(['GBM','NHA'])].copy()
# === Step 2: 构建 (enhancer, gene) -> set of cells 的映射 ===
pair_cell_map = defaultdict(set)
for _, row in df.iterrows():
    pair = (row["enhancer"], row["gene"])
    pair_cell_map[pair].add(row["cell"])

# === Step 3: 提取只出现在一个细胞的 pair，分类保存到每个细胞 ===
cell_to_enhancer_gene = defaultdict(set)
for pair, cells in pair_cell_map.items():
    if len(cells) == 1:
        cell = next(iter(cells))  # 唯一的细胞
        cell_to_enhancer_gene[cell].add(pair)

# === Step 4: 输出结果到每个细胞单独的文件 ===
output_dir = "sGBMvsNHA"
os.makedirs(output_dir, exist_ok=True)

for cell, pairs in cell_to_enhancer_gene.items():
    out_path = os.path.join(output_dir, f"{cell}_specific_enhancer_gene.txt")
    with open(out_path, "w") as f:
        for enhancer, gene in sorted(pairs):
            f.write(f"{enhancer}\t{gene}\t{cell}\n")

print(f"✅ 简化版完成：所有特异性 enhancer-gene 结果已保存在文件夹：{output_dir}/")

✅ 简化版完成：所有特异性 enhancer-gene 结果已保存在文件夹：sGBMvsNHA/
